<a href="https://colab.research.google.com/github/hanhluukim/replication-topic-modelling-in-embedding-space/blob/main/notebook_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Das Projekt aus dem Github klonen und in den Projektsordner**

In [1]:
#wenn die Ordner noch nicht geklont ist, soll dieser Fehler zuerst durchgeführt werden.
#!git clone https://github.com/hanhluukim/replication-topic-modelling-in-embedding-space.git

In [2]:
cd /content/replication-topic-modelling-in-embedding-space

/content/replication-topic-modelling-in-embedding-space


#**Die benötige Paketen für das Projekt mittels requirements.txt installieren**



In [3]:
# Falls die Packages noch nicht installiert wurden, 
!pip install -r "/content/replication-topic-modelling-in-embedding-space/requirements.txt"

# **Gebrauchte Paketen importieren**

In [4]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
import umap.umap_ as umap
import time
import plotly.express as px
from sklearn import cluster
from sklearn import metrics

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# **Vorverarbeitung und BOW-Repräsentationen für Textdaten durchführen**
1. Vocabular erstellen
2. BOW-Repräsentationen für allen Teildatensätzen

In [5]:
from src.preprare_dataset import TextDataLoader

In [6]:
# init TextDataLoader für die Datenquelle 20 News Groups
# Daten abrufen vom Sklearn, tokenisieren und besondere Charaktern entfernen
textsloader = TextDataLoader(source="20newsgroups", train_size=None, test_size=None)
textsloader.load_tokenize_texts("20newsgroups")

loading texts: ...
finished load!


In [7]:
# Beispiel von Textdaten
textsloader.show_example_raw_texts(n_docs=2)

check some sample texts of the dataset
['From', ':', 'lerxst', '@', 'wam', '.', 'umd', '.', 'edu', '(', "where's", 'my', 'thing', ')', 'Subject', ':', 'WHAT', 'car', 'is', 'this', '!', '?', 'Nntp', 'Posting', 'Host', ':', 'rac3', '.', 'wam', '.', 'umd', '.', 'edu', 'Organization', ':', 'University', 'of', 'Maryland', ',', 'College', 'Park', 'Lines', ':', '15', 'I', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'I', 'saw', 'the', 'other', 'day', '.', 'It', 'was', 'a', '2', 'door', 'sports', 'car', ',', 'looked', 'to', 'be', 'from', 'the', 'late', '60s', '/', 'early', '70s', '.', 'It', 'was', 'called', 'a', 'Bricklin', '.', 'The', 'doors', 'were', 'really', 'small', '.', 'In', 'addition', ',', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', '.', 'This', 'is', 'all', 'I', 'know', '.', 'If', 'anyone', 'can', 'tellme', 'a', 'model', 'name', ',', 'engine', 'specs', ',', 'years', 'of', 'production'

In [8]:
# Vorverarbeitung von Daten mit folgenden Schritten:
textsloader.preprocess_texts(length_one_remove=True, punctuation_lower = True, stopwords_filter = True)

start: preprocessing: ...
finised: preprocessing!


In [9]:
# Daten zerlegen für Train, Test und Validation. Erstellen Vocabular aus dem Trainset
textsloader.split_and_create_voca_from_trainset(max_df=0.7, min_df=10, stopwords_remove_from_voca=True)

permuted indices for the train set: [117  17 208 132 146 204  32 178 147 122 173 152 186  50  84]
start creating vocabulary ...
length of the vocabulary: 348
sample ten words of the vocabulary: ['nice', 'technical', 'set', 'good', 'mike', 'day', 'understanding', 'windows', 'side', 'found']
length word2id list: 348
length id2word list: 348
finished: creating vocabulary


In [10]:
# Erstellen BOW-Repräsentation für ETM Modell
for_lda_model = False 
word2id, id2word, train, test, val = textsloader.create_bow_and_savebow_for_each_set(for_lda_model=for_lda_model)

length train-documents-indices : 4616
length of the vocabulary: 348


start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
max word-id: 347
min word-id: 0
max doc-id: 149
min doc-id: 0
all docs: 4616
all words: 4616
docidx unique 150
words unique: 348
ndocs: 150
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
max word-id: 347
min word-id: 0
max doc-id: 49
min doc-id: 0
all docs: 1465
all words: 1465
docidx unique 50
words unique: 325
ndocs: 50
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [2, 3, 4, 7, 8, 10, 11, 12, 14, 15]
max word-id: 344
min word-id: 2
max doc-id: 49
min doc-id: 0
all docs: 719
all words: 719
docidx unique 50
words unique: 240
ndocs: 50
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5

# **Vocabular und IDs anzeigen als Beispiel**

---





In [11]:
# show for samples: 100 word2id and id2 word
word2id_df_100 = pd.DataFrame()
word2id_df_100['word'] = list(word2id.keys())[:100]
word2id_df_100['id'] = list(word2id.values())[:100]
word2id_df_100

,word,id
0,nice,0
1,technical,1
2,set,2
3,good,3
4,mike,4
...,...,...
95,apr,95
96,nntp,96
97,wanted,97
98,give,98


# **Die Größe von Datensätzen kontrollieren**

In [12]:
# Kontrollieren die Größen von verschiedenen Datensätzen
print(f'Size of the vocabulary after prprocessing ist: {len(textsloader.vocabulary)}')
print(f'Size of train set: {len(train["tokens"])}')
print(f'Size of val set: {len(val["tokens"])}')
print(f'Size of test set: {len(test["test"]["tokens"])}')

Size of the vocabulary after prprocessing ist: 348
Size of train set: 150
Size of val set: 100
Size of test set: 50


# **Dokumenten wiederstellen für Word2Vec Embedding**

---



In [13]:
# re-erstellen von Dokumenten nach der Vorverarbeitungen. Die Dokumenten sind in Wörtern und werden für Word-Embedding Training benutzt
docs_tr, docs_t, docs_v = textsloader.get_docs_in_words_for_each_set()
train_docs_df = pd.DataFrame()
train_docs_df['text-after-preprocessing'] = [' '.join(doc) for doc in docs_tr[:100]]
train_docs_df

,text-after-preprocessing
0,chip key systems corporation usa distribution ...
1,cc summary university nntp posting host cc new...
2,opinion distribution keywords article writes a...
3,sale price nntp posting host university distri...
4,cs windows keywords windows computer science d...
...,...
95,cc university technology computing distributio...
96,wanted nntp posting host institute distributio...
97,university department article writes article w...
98,net nntp posting host network writes question ...


# **Word-Embedding trainieren mit dem Traindatensatz**

In [14]:
from src.embedding import WordEmbeddingCreator
save_path = Path.joinpath(Path.cwd(), "vocab_embedding.txt")
wb_creator = WordEmbeddingCreator(model_name="cbow", documents = docs_tr, save_path= save_path)
wb_creator.train(min_count=2, embedding_size= 10)
vocab = list(word2id.keys())
wb_creator.create_and_save_vocab_embedding(vocab, save_path)

word-embedding train begins
word-embedding train finished
length of vocabulary from word-embedding model 347
length of the vocabulary of prepraring-dataset-vocabulary: 348


100%|██████████| 347/347 [00:00<00:00, 43265.96it/s]


True

In [15]:
v = list(wb_creator.model.wv.vocab)[0]
vec = list(wb_creator.model.wv.__getitem__(v))
print(f'word-embedding of the word-- {v}: ')
print(f'vector: {vec}')
print(f'dim of vector: {len(vec)}')

word-embedding of the word-- chip: 
vector: [0.03534068, 0.0077094086, -0.058539383, 0.047281053, 0.116620034, 0.121067256, 0.011901835, 0.070823036, 0.08540536, -0.011062479]
dim of vector: 10


# **Word-Embeddings visualieren als Beispiel**


In [16]:
# read word-embedding files
with open(save_path) as f:
  lines = f.readlines()
embedding_data = []
words_data = []
for t in lines:
  w = t.split("\t")[0]
  v = [float(e) for e in t.split("\t")[1].split(" ")]
  words_data.append(w)
  embedding_data.append(v)

In [17]:
# clustering words with KMeans and Words-Vectors
kmeans = cluster.KMeans(n_clusters=10)
kmeans.fit(embedding_data)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)

Cluster id labels for inputted data
[7 8 6 1 7 6 3 9 5 3 9 9 2 8 2 3 1 6 7 5 0 1 9 2 9 2 9 2 0 5 6 1 4 3 6 7 0
 5 7 9 7 2 2 3 6 2 5 1 8 3 2 7 6 9 6 8 6 9 2 9 1 7 2 6 6 1 0 2 8 6 9 8 4 8
 6 2 9 8 8 6 2 2 0 8 6 7 1 4 1 1 0 2 4 9 8 8 9 0 6 1 6 5 4 2 5 2 7 4 2 8 9
 2 4 4 7 5 0 8 8 1 7 8 4 7 1 5 2 6 6 7 2 1 0 3 4 2 2 1 1 5 5 2 3 9 6 5 3 5
 4 8 9 3 5 6 1 9 5 6 6 9 6 4 1 6 2 6 9 6 1 8 2 1 9 9 6 2 3 0 9 6 8 2 3 5 5
 5 0 9 7 1 4 6 8 5 6 1 6 4 2 1 8 6 5 1 9 2 5 6 4 2 5 4 7 5 3 8 8 4 1 7 5 4
 7 7 0 4 4 6 8 9 4 1 1 1 8 4 4 1 2 8 4 8 8 6 4 6 5 8 1 9 0 4 2 2 1 5 5 6 6
 5 1 4 6 7 9 4 4 8 8 4 8 1 1 0 4 5 0 8 9 1 7 6 1 9 2 2 6 4 6 2 0 4 8 7 6 6
 7 0 7 6 4 8 1 4 4 9 1 1 7 3 1 4 1 2 4 8 4 4 8 4 8 8 7 5 4 8 7 2 7 5 0 1 7
 4 2 4 9 4 4 1 6 8 6 4 6 1 1]
Centroids data
[[ 2.38457717e-02 -2.87649394e-02 -9.51878494e-02  7.68354576e-02
   1.41570863e-01  1.58163606e-01  2.73700593e-02  3.20801320e-02
   7.17820379e-02  1.60045512e-02]
 [ 5.42278618e-03 -1.84533587e-02 -2.81766511e-02  3.04420439e-02
   3.744131

In [18]:
# dimension reduction with umap
start = time.time()
reducer = umap.UMAP(random_state=42,n_components=3)
embedding = reducer.fit_transform(embedding_data)
print('Duration: {} seconds'.format(time.time() - start))

# show samples after dim-reduction in dataframe
wb = pd.DataFrame(embedding, columns=['x', 'y', 'z'])
wb['word'] = words_data
wb['cluster'] = ['cluster ' + str(c) for c in labels]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


Duration: 18.56798505783081 seconds


In [19]:
# visualization
fig = px.scatter_3d(wb, 
                    text = wb['word'],
                    x='x', y='y', z='z',
                    color = wb['cluster'],
                    title ="word-embedding-samples")
fig.show()